In [334]:
def f(loc, suff=None, f=None):
    with open(loc) as file:
        if f:
            return list(map(lambda l: f(l.rstrip()), file.readlines()))
        else:
            return list(map(lambda l: l.rstrip(), file.readlines()))
        
def g(day, suff=None, f=None):
    with open("/Users/maarten/Downloads/day{}.txt".format(day)) as file:
        if f:
            return list(map(lambda l: f(l.rstrip()), file.readlines()))
        else:
            return list(map(lambda l: l.rstrip(), file.readlines()))
def p(f):
    return "/Users/maarten/Downloads/{}.txt".format(f)

In [432]:
def dotsAndFolds(file):
    dots = set([])
    folds = []
    with open(file) as f:
        inFolds = False
        for line in f:
            if len(line.rstrip()) == 0:
                inFolds = True
                continue
            if inFolds:
                ins,pos = tuple(line.rstrip().split("="))
                folds.append((ins[-1], int(pos)))
            else:
                dots.add(tuple(map(int, line.rstrip().split(","))))
    return (dots,folds)

def fold(fold, dots):
    if fold[0] == "y":
        select = lambda d: d[1] > fold[1]
        transform = lambda d: (d[0], 2*fold[1]-d[1])
    else:
        select = lambda d: d[0] > fold[1]
        transform = lambda d: (2*fold[1]-d[0], d[1])
    newdots = dots.copy()
    for d in filter(select, dots):
        newdots.remove(d)
        newdots.add(transform(d))
    return newdots

def day13(file):
    (dots, folds) = dotsAndFolds(file)
    dots = fold(folds[0], dots)
    return len(dots)

def day13b(file):
    (dots, folds) = dotsAndFolds(file)
    for f in folds:
        dots = fold(f, dots)
    x0,x1 = min(dots, key = lambda t: t[0])[0],max(dots, key = lambda t: t[0])[0]
    y0,y1 = min(dots, key = lambda t: t[1])[1],max(dots, key = lambda t: t[1])[1]
    for y in range(y0,y1+1):
        s = ""
        for x in range(x0,x1+1):
            if (x,y) in dots: 
                s += "#"
            else:
                s += "."
        print(s)

day13b(p("day13"))

###..#....#..#...##.###..###...##...##.
#..#.#....#.#.....#.#..#.#..#.#..#.#..#
###..#....##......#.#..#.###..#..#.#...
#..#.#....#.#.....#.###..#..#.####.#.##
#..#.#....#.#..#..#.#.#..#..#.#..#.#..#
###..####.#..#..##..#..#.###..#..#..###


In [407]:
def isSmall(cave):
    return cave.lower() == cave

def walksb(s, nodes, path = "", visited = set([])):
    w = set([])
    caves = nodes[s]
    total = 0
    path = path + s
    for cave in caves:
        isDouble = False
        if isSmall(cave) and cave in visited: # small cave constraint
            if "double" in visited or cave == "start" or cave == "end":
                continue 
            else:
                isDouble = True
        if (path + cave) in w: # already captured this path
            continue
        if cave == "end":
            total += 1
        else:
            lvisited = visited.copy()
            lvisited.add(cave)
            if isDouble:
                lvisited.add("double")
            lpath = path + cave
            total += walksb(cave, nodes, lpath, lvisited)
    return total
            
            
def day12b(file):
    nodes = {}
    with open(file) as f:
        for line in f:
            e1,e2 = tuple(line.rstrip().split("-"))
            if e1 in nodes:
                nodes[e1].add(e2)
            else:
                nodes[e1]=set([e2])
            if e2 in nodes:
                nodes[e2].add(e1)
            else:
                nodes[e2]=set([e1])
    s = "start"
    w = walksb(s, nodes, "", set(["start"]))
    return w

day12b(p("day12"))
       
            

96528

In [402]:
def isSmall(cave):
    return cave.lower() == cave

def walks(s, nodes, path = "", visited = set([])):
    w = set([])
    caves = nodes[s]
    total = 0
    path = path + s
    for cave in caves:
        if isSmall(cave) and cave in visited: # small cave constraint
            continue 
        if (path + cave) in w: # already captured this path
            continue
        if cave == "end":
            total += 1
        else:
            lvisited = visited.copy()
            lvisited.add(cave)
            lpath = path + cave
            total += walks(cave, nodes, lpath, lvisited)
    return total
            
            
def day12(file):
    nodes = {}
    with open(file) as f:
        for line in f:
            e1,e2 = tuple(line.rstrip().split("-"))
            if e1 in nodes:
                nodes[e1].add(e2)
            else:
                nodes[e1]=set([e2])
            if e2 in nodes:
                nodes[e2].add(e1)
            else:
                nodes[e2]=set([e1])
    s = "start"
    w = walks(s, nodes, "", set(["start"]))
    return w

day12(p("day12"))
       
            

3450

In [377]:
def neighbours(pos, m):
    neighbours = []
    for drow in range(-1, 2):
        for dcol in range(-1, 2):
            if not (drow == 0 and dcol == 0) and drow+pos[0] >= 0 and drow+pos[0]<len(m[0]) and dcol+pos[1] >= 0 and dcol+pos[1]<len(m):
                neighbours.append((pos[0]+drow, pos[1]+dcol))
    return neighbours

def out(m):
    print("\n".join(map(lambda r: "".join(map(str, r)), m)))

def day11(file):
    m = []
    with open(file) as f:
        for line in f:
            m.append(list(map(int, list(line.rstrip()))))
    flashes = 0

    for _ in range(100):
        flashed = set([])
        flashing = set([])
        for row in range(len(m)):
            for col in range(len(m[row])):
                m[row][col] += 1
                if m[row][col] > 9:
                    flashing.add((row,col))
        while len(flashing) > 0:
            (row,col) = flashing.pop()
            n = neighbours((row,col), m)
            for nrow,ncol in n:
                m[nrow][ncol] += 1
                if m[nrow][ncol] > 9 and (nrow,ncol) not in flashed and (nrow,ncol) not in flashing:
                    flashing.add((nrow,ncol))
            flashed.add((row,col))
        for row,col in flashed:
            m[row][col] = 0
            flashes += 1
    return flashes
day11(p("day11"))


def day11b(file):
    m = []
    with open(file) as f:
        for line in f:
            m.append(list(map(int, list(line.rstrip()))))
    flashes = 0
    steps = 0
    while True:
        flashed = set([])
        flashing = set([])
        for row in range(len(m)):
            for col in range(len(m[row])):
                m[row][col] += 1
                if m[row][col] > 9:
                    flashing.add((row,col))
        steps += 1            
        while len(flashing) > 0:
            (row,col) = flashing.pop()
            n = neighbours((row,col), m)
            for nrow,ncol in n:
                m[nrow][ncol] += 1
                if m[nrow][ncol] > 9 and (nrow,ncol) not in flashed and (nrow,ncol) not in flashing:
                    flashing.add((nrow,ncol))
            flashed.add((row,col))
        if len(flashed) == len(m)*len(m[0]) or steps > 1000:
            return steps
        for row,col in flashed:
            m[row][col] = 0
            flashes += 1
day11b(p("day11"))

471

In [335]:
def day10b(file):
    terms = {')': 1, ']': 2, '}': 3, '>': 4}
    match = {')': '(', ']': '[', '}': '{', '>': '<'}
    inv = {k:v for v,k in match.items()}
    
    with open(file) as f:
        scores = []
        for line in f:
            stack = []
            for c in line.rstrip():
                if c in terms:
                    if stack[-1] == match[c]:
                        stack.pop()
                    else:
                        break # illegal
                else:
                    stack.append(c)
            else:
                if(len(stack)) == 0:
                    continue
                # legal line
                score = 0
                for c in stack[-1::-1]:
                    score *= 5
                    score += terms[inv[c]]
                scores.append(score)
        scores = sorted(scores)
        return scores[int((len(scores))/2)]
day10b(p("subsystem14"))

4874

In [336]:
def day10(file):
    scores = {')': 3, ']': 57, '}': 1197, '>': 25137}
    match = {')': '(', ']': '[', '}': '{', '>': '<'}

    with open(file) as f:
        illegal = 0
        for line in f:
            stack = []
            for c in line:
                if c in scores:
                    if stack[-1] == match[c]:
                        stack.pop()
                    else:
                        illegal += scores[c]
                        break
                else:
                    stack.append(c)
        return illegal
day10(p("day10"))

394647

In [337]:
def basin(start, m):
    result = set([start])
    while True:
        n = set([])
        for (row,col) in result:
            n.update(neighbours(row,col,m))
        n = n.difference(result)
        n = set([l for l in n if m[l[0]][l[1]] != 9])
        if len(n) == 0:
            return result
        newresult = result.union(n)
        if len(newresult) == len(result):
            return result
        result = newresult
    return result

def neighbours(row,col,m):
    return [(max(0,row-1), col), (min(len(m)-1,row+1),col), (row, max(0,col-1)), (row, min(len(m[0])-1,col+1))]

def day92(file):
    m = []
    with open(file) as f:
        for line in f:
            m.append(list(map(int, list(line.rstrip()))))
    minima = []
    for row in range(len(m)):
        for col in range(len(m[0])):
            p = m[row][col]
            localMinimum = True
            for n in neighbours(row,col,m):
                nrow = n[0]
                ncol = n[1]
                if nrow == row and ncol == col:
                    continue
                if m[nrow][ncol]<=p:
                    localMinimum = False
                    break
            if (localMinimum):
                minima.append((row,col))
    basins = []
    for (row,col) in minima:
        # flood fill up to 
        basins.append(len(basin((row,col), m)))
    
    result = 1
    for n in sorted(basins, reverse=True)[0:3]:
        result *= n
    return result
day92(p("day9"))

987840

In [338]:
def day9(file):
    m = []
    with open(file) as f:
        for line in f:
            m.append(list(map(int, list(line.rstrip()))))
    risk = 0
    for row in range(len(m)):
        for col in range(len(m[0])):
            p = m[row][col]
            localMinimum = True
            for n in [(max(0,row-1), col), (min(len(m)-1,row+1),col), (row, max(0,col-1)), (row, min(len(m[0])-1,col+1))]:
                nrow = n[0]
                ncol = n[1]
                if nrow == row and ncol == col:
                    continue
                if m[nrow][ncol]<=p:
                    localMinimum = False
                    break
            if (localMinimum):
                risk += p+1
    return risk
day9(p("caves06"))

1034

In [339]:
def day82(file):
    easy = {1: "cf", 4: "bcdf", 7: "acf", 8: "abcdefg"}
    hard = {0: "abcefg", 2: "acdeg", 3: "acdfg", 5: "abdfg", 6: "abdefg", 9: "abcdfg"}
    with open(file) as f:
        total = 0
        for line in f:
            ip,op = tuple(line.rstrip().split(' | '))
            inputs = ip.split(' ')
            outputs = op.split(' ')
            mutations = {i:set(list("abcdefg")) for i in "abcdefg"}
            for inp in inputs:
                for d,v in easy.items():
                    if len(inp) == len(v):
                        # found a match: add pattern
                        for c in v:
                            mutations[c] = mutations[c].intersection(set(list(inp)))
                if len(inp) == 5:
                    mutations['a'] = mutations['a'].intersection(set(list(inp)))
                    mutations['d'] = mutations['d'].intersection(set(list(inp)))
                    mutations['g'] = mutations['g'].intersection(set(list(inp)))
                if len(inp) == 6:
                    mutations['a'] = mutations['a'].intersection(set(list(inp)))
                    mutations['b'] = mutations['b'].intersection(set(list(inp)))
                    mutations['f'] = mutations['f'].intersection(set(list(inp)))
                    mutations['g'] = mutations['g'].intersection(set(list(inp)))
            while not all(map(lambda s: len(s) == 1, mutations.values())):
                for k, v in filter(lambda s: len(s[1]) == 1, mutations.items()):
                    for mk, mv in mutations.items():
                        if mutations[mk] != v:
                            mutations[mk] = mutations[mk].difference(v)
            inv_map = {v: k for k, v in list(hard.items()) + list(easy.items())}
            inv_mut = {list(v)[0]: k for k, v in mutations.items()}
            out_num = ""
            for out in outputs:
                s = ""
                for k in out:
                    s += inv_mut[k]
                out_num += str(inv_map["".join(sorted(s))])
            total += int(out_num)
    return total
day82(p("day8"))

1055164

In [340]:
def day8(file):
    digits = {1: "cf", 4: "bcdf", 7: "acf", 8: "abcdefg"}
    with open(file) as f:
        total = 0
        for line in f:
            ip,op = tuple(line.rstrip().split(' | '))
            inputs = ip.split(' ')
            outputs = op.split(' ')
            for inp in inputs:
                for d,v in digits.items():
                    if len(inp) == len(v):
                        for out in outputs:
                            if len(out) == len(v):
                                total += 1
    return total
day8(p("day8"))

495

In [341]:
def day72(file):
    with open(file) as f:
        crabs = list(map(int, f.readline().rstrip().split(',')))
    m = None
    pos = None
    for i in range(max(crabs)):
        delta = sum(map(lambda c: abs(c-i)*(abs(c-i)+1)/2, crabs))
        if m is None or delta < m:
            m = delta
            pos = i
    return int(m)
day72(p("day7"))

100727924

In [342]:
def day7(file):
    with open(file) as f:
        crabs = list(map(int, f.readline().rstrip().split(',')))
    m = None
    pos = None
    for i in range(max(crabs)):
        delta = sum(map(lambda c: abs(c-i), crabs))
        if m is None or delta < m:
            m = delta
            pos = i
    return m
day7(p("day7"))

359648

In [343]:
def day62(file):
    fish = {key:0 for key in range(9)}
    with open(file) as f:
        l = f.readline()
        start = list(map(int, l.rstrip().split(',')))
        for f in start:
            fish[f] += 1
    for day in range(256):
        birthcount = fish[0]
        newfish = {key:0 for key in range(9)}
        # slide all fish to the left in age dict
        for age in range(9):
            newage = 6 if age == 0 else age - 1
            newfish[newage] += fish[age]
        # add the new births
        fish = newfish
        fish[8] = birthcount
    return sum(fish.values())
day62(p("day6"))

1681503251694

In [164]:
def day6(file):
    with open(file) as f:
        l = f.readline()
        fish = list(map(int, l.rstrip().split(',')))
    for day in range(80):
        newfish = []
        for i,f in enumerate(fish):
            if (f == 0):
                f = 6
                newfish.append(8)
            else:
                f -= 1
            fish[i] = f
        fish += newfish
    return len(fish)
day6(p("day6"))

372984

In [129]:
def day52(file):
    criteria = []
    hits = {}
    s = 0
    with open(file) as f:
        for line in f:
            line = line.rstrip()
            left,right = tuple(line.split('->'))
            x1,y1 = tuple(map(int, left.rstrip().split(',')))
            x2,y2 = tuple(map(int, right.rstrip().split(',')))
            xSign = 1 if x2 >= x1 else -1
            ySign = 1 if y2 >= y1 else -1
            xs = range(x1, x2+xSign, xSign)
            ys = range(y1, y2+ySign, ySign)
            if (x1 == x2 or y1 == y2):
                r = zip(list(xs)*len(ys),list(ys)*len(xs))
            else:
                r = zip(list(xs),list(ys))
            for x,y in r:
                if (x,y) in hits:
                    if hits[(x,y)] == 1:
                        s += 1
                    hits[(x,y)] += 1
                else:
                    hits[(x,y)] = 1
    return s
day52(p("day5"))

15463

In [109]:
def day5(file):
    criteria = []
    hits = {}
    s = 0
    with open(file) as f:
        for line in f:
            line = line.rstrip()
            left,right = tuple(line.split('->'))
            x1,y1 = tuple(map(int, left.rstrip().split(',')))
            x2,y2 = tuple(map(int, right.rstrip().split(',')))
            if not (x1 == x2 or y1 == y2):
                continue
            for x in range(min(x1, x2), max(x1, x2)+1):
                for y in range(min(y1, y2), max(y1, y2)+1):
                    if (x,y) in hits:
                        if hits[(x,y)] == 1:
                            s += 1
                        hits[(x,y)] += 1
                    else:
                        hits[(x,y)] = 1
                    
    return s
day5(p("day5"))

5698

In [87]:
def board_mark(board, number):
    for l in board:
        if number in l:
            idx = l.index(number)
            l[idx] = -10000
    return board

def score_board(board):
    s = 0
    for l in board:
        s += sum(filter(lambda n: n > 0, l))
    return s

def val_board(board):
    neg_row = filter(lambda l: all(map(lambda n: n<0, l)), board)
    neg_col = filter(lambda l: all(map(lambda n: n<0, l)), list(map(list, zip(*board))))
    return any(neg_row) or any(neg_col)
                     
def day4_2(file):
    with open(file) as f:
        draws = map(int, f.readline().split(","))
        
        boards = []
        done = set([])
        board = None
        for line in f:
            line = line.rstrip()
            if not line:
                if board:
                    boards.append(board)
                board = []
            else:
                board.append(list(map(int, line.split())))
        boards.append(board)
        
        for draw in draws:
            for i, board in enumerate(boards):
                board = board_mark(board, draw)
                if val_board(board):
                    done.add(i)
                    if len(done) == len(boards):
                        s = score_board(board)
                        return s*draw
                boards[i] = board
print(day4_2("/Users/maarten/Downloads/day4.txt"))

21070


In [75]:
def board_mark(board, number):
    for l in board:
        if number in l:
            idx = l.index(number)
            l[idx] = -10000
    return board

def score_board(board):
    s = 0
    for l in board:
        s += sum(filter(lambda n: n > 0, l))
    return s

def val_board(board):
    neg_row = filter(lambda l: all(map(lambda n: n<0, l)), board)
    neg_col = filter(lambda l: all(map(lambda n: n<0, l)), list(map(list, zip(*board))))
    return any(neg_row) or any(neg_col)
                     
def day4(file):
    with open(file) as f:
        draws = map(int, f.readline().split(","))
        
        boards = []
        board = None
        for line in f:
            line = line.rstrip()
            if not line:
                if board:
                    boards.append(board)
                board = []
            else:
                board.append(list(map(int, line.split())))
        boards.append(board)
        
        for draw in draws:
            for i, board in enumerate(boards):
                board = board_mark(board, draw)
                if val_board(board):
                    s = score_board(board)
                    return s*draw
                boards[i] = board
print(day4("/Users/maarten/Downloads/demo.txt"))

4512


In [8]:
# Day 3.2
def day3(file):
    lines = f(file)
    bits = {}
    
    mkeep = lines
    lkeep = lines
    m = 0
    l = 0
    for i in range(len(lines[0])):
        s = sum(map(lambda l: int(l[i]), mkeep))
        if s >= len(mkeep)/2:
            mkeep = list(filter(lambda l: l[i] == '1', mkeep))
        else:
            mkeep = list(filter(lambda l: l[i] == '0', mkeep))
        if len(mkeep) == 1:
            m = int(min(mkeep), 2)
        s = sum(map(lambda l: int(l[i]), lkeep))
        if s >= len(lkeep)/2 and s != len(lkeep):
            lkeep = list(filter(lambda l: l[i] == '0', lkeep))
        else:
            lkeep = list(filter(lambda l: l[i] == '1', lkeep))
        if len(lkeep) == 1:
            l = int(min(lkeep), 2)
    return m*l
        
day3("/Users/maarten/Downloads/report18.txt".format(3))

43774224

In [4]:
# Day 3.1
def day3():
    lines = f("/Users/maarten/Downloads/day{}.txt".format(3))
    bits = {}
    n = 0

    for line in lines:
        for i,bit in enumerate(line):
            if i in bits:
                bits[i]+=int(line[i])
            else:
                bits[i]=int(line[i])
        n+=1
        
    m = ""
    l = ""

    for k,v in sorted(bits.items(), key = lambda d: d[0]):
        if v<n/2:
            m += "0"
            l += "1"
        else:
            m += "1"
            l += "0"
    return int(m, 2)*int(l,2)
day3()

1307354

In [43]:
day2 = g(2)

In [44]:
#Day 2
deltas = {"forward": (1,0), "down": (0,1), "up": (0,-1)}
pos = (0,0)
for d in day2:
    inst, amount = tuple(d.split(" "))
    amount = int(amount)
    delta = deltas[inst]
    pos = (pos[0] + amount * delta[0], pos[1] + amount * delta[1])
pos[0]*pos[1]

2272262

In [41]:
#Day 2.2
aim = 0
hpos = 0
depth = 0
for d in day2:
    inst, amount = tuple(d.split(" "))
    amount = int(amount)
    if inst == "down":
        aim += amount
    if inst == "up":
        aim -= amount
    if inst == "forward":
        hpos += amount
        depth += aim * amount
hpos * depth

2134882034

In [31]:
day1 = g(1, f=int)

In [33]:
#Day 1
prev = None
cnt = 0
for d in day1:
    if prev and d > prev:
        cnt += 1
    prev = d
cnt

1390

In [35]:
#Day 1.2
prev = None
cnt = 0
for i,d in enumerate(day1):
    s = sum(day1[i:i+3])
    if prev and s > prev:
        cnt += 1
    prev = s
cnt

1457